In [7]:
from numpy.polynomial import Polynomial
from phe import paillier
from numpy import prod

In [28]:
class Player:
    """
    Class of honest-but-curious adversaries.

    Parameters
    ----------
    S : set_like
    sk : secret key to a homomorpic cryptosystem
    """
    def __init__(self, S, sk):
        self.S = S
        self.k = len(S)
        self.sk = sk
        self.f_list = []
    
    def polynomial(self):
        # cria o polinómio f a partir do conjunto S 
        self.f = prod([Polynomial([1, -s]) for s in self.S])
    
    def encrypt(self, pk):
        # encipta o polinómio f com pk
        self.ef = Polynomial([pk.encrypt(x) for x in self.f.coef])
    
    def send_f(self, encrypted_polynomial):
        # adiciona polinomio encriptado de outro player a f_list
        self.f_list.append(encrypted_polynomial)

    def phi():
        # calcula phi_i
        pass

    def lam():
        # calcula lambda_i
        pass

    def multiset():
        # for each a in S_i, he calculates b such that
        # (x - a)^b|p and (x - a)^{b+1} does not divide p
        pass

    

Funções da secção 4.2

In [ ]:
def sum(f, g):
    return

In [ ]:
def product(f, g):
    return

In [ ]:
def derivative(f, d):
    return

In [ ]:
def evaluation(f, b):
    return

Função da secção 5.1

In [ ]:
def psi(players, c, pk):
    """
    Perform a private Set-Intersection secure against a coalition of honest-but-curious
    adversaries.

    Parameters
    ----------
    players : array_like construct of class Player elements
    c : dishonestly colluding constant
    pk: public key to a homomorpic cryptosystem
    """
    assert(len(players) >= 2, "Not enough players")
    assert(c < len(players), "Dishonestly colluding too large")
    assert(all([player.k == player[0].k for player in players]),\
        "Differently sized private sets")
    #1
    for player in players:
        #a
        player.polynomial()
        #b
        player.encrypt()
        # Aqui se i + c > n e suposto voltar ao primeiro??
        for other_player in (players[i+1, i+c+1]):
            other_player.send_f(player.ef)
        #c
        # Qual é o domínio??
        # DEFAULT_KEYSIZE = 3072
        # BASE = 16
        #d
    #2
    #3
    for player in players[1:]:
        #a
        #b
        #c
    #4
    #5
    p = decrypt(p)
    #6
    for player in players:
        player.multiset(p)

In [3]:
# Generate keys
pk, sk = paillier.generate_paillier_keypair()

In [29]:
# Create players
a = Player([1,2,3],sk)
b = Player([2,3,4],sk)

In [ ]:
# Calculate private set
psi()

In [ ]:
#find private set

In [ ]:
#get obtained provate sets

In [ ]:
#compare make sure the sets are the same